## Recap and Skill Test

In this notebook, you will recap the skills you've learned about **Quantization, Parameter-Efficient Fine-Tuning (PEFT), and Unsloth** techniques. You will apply these methods to memory-efficiently finetune a lange model.

### Objectives:

1. **Quantize a Model**: Apply 4-bit quantization when loading a pre-trained model.
2. **Parameter-Efficient Fine-Tuning (PEFT)**: Use adapter layers to fine-tune a model efficiently.

## Task:

1. Choose a different pre-trained model from the Huggingface Hub (e.g., Mistral 7B Instruct).
2. Load the model using 4-bit quantization.
3. Finetune the model on a dataset of your choice (e.g., vicgalle/alpaca-gpt4).
4. Evaluate the model's performance before and after finetuning.

### Hints:

Some models' tokenizers do not come with a `pad_token`. It might be necessary to manually set the `pad_token` to some other token, e.g.:
```
tokenizer.pad_token = tokenizer.unk_token
```

In my example, I am going to use `mistralai/Mistral-7B-Instruct-v0.3`, which is a gated model. You need a Huggingface account and ask for access to the model. Then you need to get an account token from https://huggingface.co/settings/tokens and execute `huggingface-cli login` in a Terminal tab in JupyterHub.

If you want to use Mistral 7B Instruct, you can also use the following path so that we do not download the same model 50 times:
```
'/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/mistralai--Mistral-7B-Instruct-v0.3'
```
Even if you use the model from the filesystem, I would suggest that you still go to the Huggingface Hub and have a look at the list of models and datasets there. You may also create an account, if you don't have one yet, and run `huggingface-cli login`.

In [1]:
import torch
from accelerate import PartialState
from datasets import load_dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig

In [2]:
# model_id = 'mistralai/Mistral-7B-Instruct-v0.3'
model_id = '/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/mistralai--Mistral-7B-Instruct-v0.3'

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = 'right'

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    attn_implementation='sdpa',  # 'eager', 'sdpa', or "flash_attention_2"
    torch_dtype=torch.bfloat16,
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data = load_dataset('vicgalle/alpaca-gpt4', split='train')

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-6ef3991c06080e(…):   0%|          | 0.00/48.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [4]:
data

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [5]:
data[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for 

In [6]:
data = data.map(lambda entry: {'text_mistral7b': tokenizer.apply_chat_template([
    {'role': 'user', 'content': entry['instruction']},
    {'role': 'assistant', 'content': entry['output']}
], tokenize=False)})

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [7]:
data[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for 

In [8]:
peft_config = LoraConfig(
    task_type='CAUSAL_LM',
    r=16,
    lora_alpha=32,  # rule: lora_alpha should be 2*r
    lora_dropout=0.05,
    bias='none',
    target_modules='all-linear',
)

In [9]:
training_arguments = SFTConfig(
    output_dir='output/mistral7b-alpaca-gpt4',
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True, # Gradient checkpointing improves memory efficiency, but slows down training,
        # e.g. Mistral 7B with PEFT using bitsandbytes:
        # - enabled: 11 GB GPU RAM and 8 samples/second
        # - disabled: 40 GB GPU RAM and 12 samples/second
    gradient_checkpointing_kwargs={'use_reentrant': False},  # Use newer implementation that will become the default.
    optim='adamw_torch',  # 'paged_adamw_32bit' can save GPU memory
    learning_rate=2e-4,  # QLoRA suggestions: 2e-4 for 7B or 13B, 1e-4 for 33B or 65B
    warmup_steps=200,
    lr_scheduler_type='cosine',
    logging_strategy='steps',  # 'no', 'epoch' or 'steps'
    logging_steps=10,
    save_strategy='no',  # 'no', 'epoch' or 'steps'
    max_steps=100,
    bf16=True,  # mixed precision training
    report_to='none',  # disable wandb
    dataset_text_field='text_mistral7b',
    max_seq_length=1024,
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


In [10]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=data,
    peft_config=peft_config,
    processing_class=tokenizer,
)

Adding EOS to train dataset:   0%|          | 0/52002 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/52002 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/52002 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


[2025-09-05 17:44:04,830] [INFO] [real_accelerator.py:260:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-09-05 17:44:07,708] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
if hasattr(trainer.model, "print_trainable_parameters"):
    trainer.model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 7,289,966,592 || trainable%: 0.5754


In [12]:
result = trainer.train()

# Print statistics:
print(f"Run time: {result.metrics['train_runtime']:.2f} seconds")
print(f"Training speed: {result.metrics['train_samples_per_second']:.1f} samples/s")

Step,Training Loss
10,1.809800
20,1.593300
30,1.441000
40,1.398200
50,1.122100
60,1.222900
70,1.172600
80,1.114000
90,1.125300
100,1.194000


Run time: 158.05 seconds
Training speed: 5.1 samples/s


In [13]:
# trainer.save_model()